In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import sys
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import random as random

In [2]:
df2 = pd.read_csv("data_with_prop_ext.csv").iloc[:,1:]

In [3]:
 df2.shape

(163373, 25)

In [4]:
insert_day_list = list(df2["insert_month_day"].unique())

In [5]:
group_id_df = (
    df2.drop_duplicates(subset="train_id")
    .sort_values(by="train_id")
    .loc[:, ["train_id", "pair_1", "pair_2"]]
    .dropna()
    .assign(pair_0=lambda _df: _df["train_id"])
    .loc[:, ["pair_0", "pair_1", "pair_2"]]
    .drop_duplicates()
    .reset_index(drop=True)
)

In [6]:
group_id_df.shape

(5353, 3)

In [7]:
group_id_df = pd.concat(
    [
        group_id_df,
        pd.DataFrame({"group_id_list_0": [f"{x}" for x in range(1, 5354)]}),
        pd.DataFrame({"group_id_list_1": [f"{x}" for x in range(1, 5354)]}),
        pd.DataFrame({"group_id_list_2": [f"{x}" for x in range(1, 5354)]}),
    ],
    axis=1,
)

In [8]:
df2 = df2.loc[:,~df2.columns.isin(["pair_1","pair_2"])]

In [9]:
df_grouped = (
    df2.merge(
        group_id_df.loc[:, ["group_id_list_0", "pair_0"]].rename(columns = {"pair_0" : "train_id"}),
        on="train_id",
        how="left",
    ).merge(
        group_id_df.loc[:, ["group_id_list_1", "pair_1"]].rename(columns = {"pair_1" : "train_id"}),
        on="train_id",
        how="left"
    ).merge(
        group_id_df.loc[:, ["group_id_list_2", "pair_2"]].rename(columns = {"pair_2" : "train_id"}),
        on="train_id",
        how="left",
    )
    #.dropna()
    #.loc[:, list(df.columns) + ["group_id_list_0", " group_id_list_1"]]
)

## Megcsináltam a jó grouping változót. Most pedig megnézzük egy adott group mentén az egész folyamatot

In [22]:
random_group = "201"

In [23]:
df_group_rand = df_grouped.loc[
    (
        (df_grouped["group_id_list_0"] == random_group)
        | (df_grouped["group_id_list_1"] == random_group)
        | (df_grouped["group_id_list_2"] == random_group)
    )
]

In [24]:
df_rand_count = df_group_rand.drop_duplicates(subset = ["train_id","insert_month_day"]).groupby("insert_month_day").agg({"train_id" : "count"})

In [25]:
keep_insertday = df_rand_count.loc[lambda _df: _df["train_id"] == 3,:].reset_index()["insert_month_day"].to_list()

In [27]:
df_group_rand = df_group_rand.loc[
    lambda _df: _df["insert_month_day"].isin(keep_insertday),
    ~df_group_rand.columns.isin(
        ["group_id_list_1", "group_id_list_2", "group_id_list_0"]
    ),
]

In [28]:
new_prop_df = df_group_rand.iloc[:,:-3]

In [29]:
list_of_inserts = list(new_prop_df["insert_month_day"].unique())

In [30]:
replace_dict = {key:value for (key,value) in zip(insert_day_list, range(1,len(insert_day_list)+1))}

In [31]:
def prop_values_count(data, list_of_insert_dates, i, replace_dict_):
    return (
    data.loc[lambda _df: _df["insert_month_day"] == i].assign(
        sum_sales=lambda _df: _df.loc[lambda __df: __df["insert_month_day"] == i]["seats_sold"].sum(),
        prop_sales = lambda _df: (_df["seats_sold"] / _df["sum_sales"]).replace(np.nan, 0),
        group_treatment=lambda _df: True if _df["price_increase"].sum() > 0 else False,
        self_treatment = lambda _df: _df["price_increase"]
    ).drop(columns = ["sum_sales"]).replace(replace_dict).rename(columns = {"insert_month_day" : "trend"})
)

In [32]:
prop_values_count(new_prop_df, list_of_inserts, 4251, replace_dict)

,train_id,trend,mean_econ_price,max_econ_seats,min_econ_seats,mean_econ_seats,depart_month,depart_day,depart_hour,insert_date,...,price_change,price_change_percentage,price_change_direction,price_increase,price_decrease,seats_sold,depart_monthday,prop_sales,group_treatment,self_treatment


In [33]:
dfs = [prop_values_count(new_prop_df, list_of_inserts, x, replace_dict) for x in list_of_inserts]

In [34]:
new_rand_grouped_df = pd.concat(dfs)

## Most az előzőeket meg kell csinálni mindegyik groupra

In [35]:
df_grouped["group_id_list_0"].astype(float).max()

5353.0

In [36]:
group_list = [str(x) for x in range(1,5354)]

In [37]:
group_list[5352]

'5353'

In [39]:
from tqdm.notebook import tqdm

In [40]:
dataframes = []
for group in tqdm(group_list):
    # print(group)
    df_group = df_grouped.loc[
        (
            (df_grouped["group_id_list_0"] == group)
            | (df_grouped["group_id_list_1"] == group)
            | (df_grouped["group_id_list_2"] == group)
        )
    ]
    df_rand_count = (
        df_group.drop_duplicates(subset=["train_id", "insert_month_day"])
        .groupby("insert_month_day")
        .agg({"train_id": "count"})
        .reset_index()
    )
    keep_insertday = df_rand_count.loc[lambda _df: _df["train_id"] == 3, :][
        "insert_month_day"
    ].to_list()

    new_prop_df = df_group.loc[
        lambda _df: _df["insert_month_day"].isin(keep_insertday),
        ~df_group.columns.isin(
            ["group_id_list_1", "group_id_list_2", "group_id_list_0"]
        ),
    ].iloc[:, :-3]

    list_of_inserts = list(new_prop_df["insert_month_day"].unique())

    dfs_bygroup = [
        prop_values_count(new_prop_df, list_of_inserts, x, replace_dict)
        for x in list_of_inserts
    ]
    dataframes.append(pd.concat(dfs_bygroup))

  0%|          | 0/5353 [00:00<?, ?it/s]

In [41]:
len(dataframes)

5353

In [42]:
index_range = range(len(dataframes))

df_dict = {index : frame for (index, frame) in zip(index_range,dataframes )}

## Treatment indicator making

In [43]:
import altair as alt

In [44]:
test_df = df_dict[250]

In [45]:
test_df["group_treatment"] = test_df["group_treatment"].replace({True : 1, False : 0})
test_df["self_treatment"] = test_df["self_treatment"].replace({True : 1, False : 0})
test_df = test_df.assign(treatment_in_period = lambda _df: _df["group_treatment"] + _df["self_treatment"])
test_df["treatment_in_period"] = test_df["treatment_in_period"].replace(2,0)

In [47]:
price_evolution = (
    alt.Chart(test_df)
    .mark_line()
    .encode(
        x="trend",
        y="mean_econ_price",
        color="train_id",
        strokeDash="train_id",
    )
    .properties(height = 300, width = 250)
    .interactive()
)
seat_evolution = (
    alt.Chart(test_df)
    .mark_line()
    .encode(
        x="trend",
        y="mean_econ_seats",
        color="train_id",
        strokeDash="train_id",
    )
    .properties(height = 300, width = 250)
    .interactive()
)

vertline_1 = alt.Chart().mark_rule(strokeDash = [1,3]).encode(
    x='a:Q'
)
vertline_2 = alt.Chart().mark_rule(strokeDash = [1,3]).encode(
    x='b:Q'
)
vertline_3 = alt.Chart().mark_rule(strokeDash = [1,3]).encode(
    x='c:Q'
)

vertline_4 = alt.Chart().mark_rule(strokeDash = [1,3]).encode(
    x='d:Q'
)
seat_line_evolution = (
    alt.layer(seat_evolution, vertline_1, vertline_2, data=test_df)
    .transform_calculate(a="4",b = "19")
    .facet(row="site")
)

price_line_evolution = (
    alt.layer(price_evolution, vertline_3, vertline_4, data=test_df)
    .transform_calculate(c = "4", d = "19")
    .facet(row="site")
)

In [48]:
price_evolution | seat_line_evolution

alt.HConcatChart(...)

In [49]:
def diff(li1, li2):
    return list(set(li1) - set(li2)) + list(set(li2) - set(li1))

In [50]:
def define_treatment_intervals(df):

    self = df.index[df["self_treatment"] == 1].tolist()
    csoport = df.index[df["group_treatment"] == 1].tolist()
    last_index = df.index[-1]

    origin_only_group_treatment = diff(self, csoport)
    only_group_treatment_list = diff(self, csoport)
    # diff után, listába rakom a dolgokat
    self_treatment_list = list(self)

    # végigiterálok mindkettő dolgon
    for i in range(len(self_treatment_list)):
        for x in range(len(only_group_treatment_list)):
            try:
                # belerakom azt amelyik ketté közé esik
                if (
                    self_treatment_list[i] > only_group_treatment_list[x]
                    and self_treatment_list[i] < only_group_treatment_list[x + 1]
                ):
                    only_group_treatment_list.append(self_treatment_list[i])
            except IndexError:
                pass
            # belerakom az utolsót, indexerror miatt kiesne
            if self_treatment_list[i] > only_group_treatment_list[-1]:
                only_group_treatment_list.append(self_treatment_list[i])

    x_teljes = diff(only_group_treatment_list, origin_only_group_treatment) + list(
        set(origin_only_group_treatment)
    )
    x1 = diff(only_group_treatment_list, origin_only_group_treatment)
    x2 = list(set(origin_only_group_treatment))
    x_teljes.sort()
    list_final = []
    for x in range(len(x_teljes)):
        try:
            if x_teljes[x] in x1:
                pass
            # elif (x_teljes[x] in x2) and (x_teljes[x+1] in x2):
            #    pass
            elif (x_teljes[x] in x1) and (x_teljes[x + 1] in x1):
                pass
            else:
                list_final.append(range(x_teljes[x], x_teljes[x + 1]))
        except IndexError:
            # print("we are on the last index")
            list_final.append(range(x_teljes[x], last_index + 1))
    return list_final

In [52]:
def change_treatment_variable(dataframe):
    list_of_3_dfs = []
    for train in list(dataframe["train_id"].unique()):
        df_train = dataframe.loc[lambda _df: _df["train_id"] == train, :].reset_index()
        for ranges in range(len(define_treatment_intervals(df_train))):
            df_train.loc[
                define_treatment_intervals(df_train)[ranges], "treatment_in_period"
            ] = 1
        list_of_3_dfs.append(df_train)
    return pd.concat(list_of_3_dfs)

In [53]:
len(df_dict)

5353

In [54]:
df_lista_megint = []
for i in tqdm(range(len(df_dict))):
    df_dict[i]["group_treatment"] = df_dict[i]["group_treatment"].replace(
        {True: 1, False: 0}
    )
    df_dict[i]["self_treatment"] = df_dict[i]["self_treatment"].replace(
        {True: 1, False: 0}
    )
    df_dict[i] = df_dict[i].assign(
        treatment_in_period=lambda _df: _df["group_treatment"] + _df["self_treatment"]
    )
    df_dict[i]["treatment_in_period"] = df_dict[i]["treatment_in_period"].replace(2, 0)
    df_lista_megint.append(change_treatment_variable(df_dict[i]))

  0%|          | 0/5353 [00:00<?, ?it/s]

In [55]:
index_range = range(len(df_lista_megint))

df_dict_ver2 = {index : frame for (index, frame) in zip(index_range,df_lista_megint )}

In [56]:
len(df_dict_ver2)

5353

In [57]:
import pickle

In [58]:
with open('dict_of_dfs_ver2_full.pickle', 'wb') as f:
    pickle.dump(df_dict_ver2, f)